# HW 5. Пусть бот заговорит. Задание DaNetQA из Russian SuperGLUE  

## Загрузка данных

In [1]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [2]:
!pip install transformers

In [3]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек
import time          # Библиотека для времени
import random

import warnings
warnings.filterwarnings("ignore")

random.seed(42)
np.random.seed(42)

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import json
import random
from tqdm import tqdm
from sklearn.metrics import *

random.seed(42)
np.random.seed(42)


In [5]:
import torch

from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import get_scheduler


In [6]:
# https://russiansuperglue.com/tasks/
# Yes/no Question Answering Dataset for the Russian

!wget --no-check-certificate https://russiansuperglue.com/tasks/download/DaNetQA

--2024-02-17 19:30:24--  https://russiansuperglue.com/tasks/download/DaNetQA
Resolving russiansuperglue.com (russiansuperglue.com)... 149.100.138.62
Connecting to russiansuperglue.com (russiansuperglue.com)|149.100.138.62|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 1293761 (1.2M) [application/zip]
Saving to: ‘DaNetQA’

DaNetQA             100%[===================>]   1.23M  2.42MB/s    in 0.5s    

2024-02-17 19:30:25 (2.42 MB/s) - ‘DaNetQA’ saved [1293761/1293761]



In [7]:
!unzip DaNetQA -d QA

Archive:  DaNetQA
   creating: QA/DaNetQA/
  inflating: QA/DaNetQA/train.jsonl  
   creating: QA/__MACOSX/
   creating: QA/__MACOSX/DaNetQA/
  inflating: QA/__MACOSX/DaNetQA/._train.jsonl  
  inflating: QA/DaNetQA/.DS_Store    
  inflating: QA/__MACOSX/DaNetQA/._.DS_Store  
  inflating: QA/DaNetQA/test.jsonl   
  inflating: QA/__MACOSX/DaNetQA/._test.jsonl  
  inflating: QA/DaNetQA/val.jsonl    
  inflating: QA/__MACOSX/DaNetQA/._val.jsonl  
  inflating: QA/__MACOSX/._DaNetQA   


In [8]:
train = pd.read_json('/content/QA/DaNetQA/train.jsonl', orient='records', lines = True)
val = pd.read_json('/content/QA/DaNetQA/val.jsonl', orient='records', lines = True)
test = pd.read_json('/content/QA/DaNetQA/test.jsonl', orient='records', lines = True)

In [9]:
train

,question,passage,label,idx
0,Вднх - это выставочный центр?,«Вы́ставочный центр» — станция Московского мон...,True,0
1,Вднх - это выставочный центр?,"Вы́ставка достиже́ний наро́дного хозя́йства ,...",True,1
2,Был ли джиган в black star?,Вместе с этим треком они выступили на церемони...,True,2
3,Xiaomi конкурент apple?,"Xiaomi — китайская компания, основанная в 2010...",True,3
4,Был ли автомат калашникова в вов?,Отметив некоторые недостатки и в целом удачную...,False,4
...,...,...,...,...
1744,Разрешен ли такой вид ловли акул в настоящее в...,Для человека они потенциально полезны в медици...,True,1745
1745,Закреплено ли Гражданство в Конституции,Гражданство является одним из институтов конст...,True,1746
1746,"Существуют ли примеры, когда не совсем достато...",В философии под эффективностью понимается спос...,True,1747
1747,Решен ли вопрос о подлинности Диалога Тацита?,В XIX веке Диалог считали первым произведением...,False,1748


In [10]:
# Разделяем два предложения специальным символом [SEP]
from collections import Counter

train_X = train['question'] + '[SEP]' + train['passage']
train_y = train['label'].astype(int)
val_X = val['question'] + '[SEP]' + val['passage']
val_y = val['label'].astype(int)
test_X = test['question'] + '[SEP]' + test['passage']

print('Train size:', len(train_X))
print('Val size:', len(val_X))
print('Test size:', len(test_X))
print('\n')
print('Train labels counts\n', Counter(train_y), '\n')
print('Eval labels counts\n', Counter(val_y), '\n')



Train size: 1749
Val size: 821
Test size: 805


Train labels counts
 Counter({1: 1061, 0: 688}) 

Eval labels counts
 Counter({1: 412, 0: 409}) 



## Обучение Train Loop

In [11]:
class TrainDataset(Dataset):

    def __init__(self, X, label, max_len):
        self.text = X.reset_index(drop=True)
        self.label = label.reset_index(drop=True)
        self.max_len = max_len

    def tokenize(self, text):
        return tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=self.max_len)

    def __len__(self):
        return self.label.shape[0]

    def __getitem__(self, index):
        output = self.text[index]
        output = self.tokenize(output)
        output.update({'labels': torch.tensor(self.label[index])})
        return {k: v.reshape(-1).to(device) for k, v in output.items()}

class TestDataset(Dataset):

    def __init__(self, X, max_len):
        self.text = X.reset_index(drop=True)
        self.max_len = max_len

    def tokenize(self, text):
        return tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=self.max_len)

    def __len__(self):
        return self.text.shape[0]

    def __getitem__(self, index):
        output = self.text[index]
        output = self.tokenize(output)
        return {k: v.reshape(-1).to(device) for k, v in output.items()}



In [12]:
def train_model(train_dataloader, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1} / {num_epochs} \n -------------------')
        for n_batch, batch in enumerate(train_dataloader):
            # Forward pass (скормить данные в нейросеть и пробросить вперед)
            outputs = model(**batch)
            loss = outputs.loss
            if n_batch % 50 == 0:
                loss_value, current = loss.item(), n_batch * batch['input_ids'].shape[0]
                print(f"Loss train: {loss_value:>7f}  [{current:>5d}/{len(train_ds):>5d}]")
                print('Evaluating...')
                preds, true = test_model(eval_dataloader, eval=True)
                print(f'F1-score = {f1_score(preds, true, average="macro"):>3f}\n')
            # Backward pass (backpropagation - посчитать градиенты по всем параметрам с помощью обратного распространения ошибки)
            loss.backward()
            # Обновить параметры с помощью optimizer.step()
            optimizer.step()
            lr_scheduler.step()
            # занулить градиенты предыдущего шага
            optimizer.zero_grad()

def test_model(test_dataloader, eval=False):
    model.eval()
    y_pred = np.array([])
    y_true = np.array([])
    for n_batch, batch in enumerate(test_dataloader):
        if eval:
            y_true = np.hstack([y_true, batch['labels'].cpu().numpy().reshape(-1)])
        outputs = model(**batch)
        y_pred = np.hstack([y_pred, outputs['logits'].argmax(axis=1).detach().cpu().numpy()])
    return y_pred, y_true

In [13]:
def get_reply(text1, text2):
  with torch.no_grad():
    model.eval()
    input_msg = tokenizer(text1, text2, return_tensors='pt', padding='max_length', truncation=True, max_length=max_len)
    input_msg = input_msg.to(device)
    outputs = model(**input_msg)[0]
    output = outputs.argmax(axis=1).detach().cpu().numpy()
    probabilities = torch.softmax(outputs, dim=-1).detach().cpu().numpy()
    #output = le.inverse_transform(output)
    print(text1, output, probabilities.max())
    #return output



## Модель ruBert

In [12]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruBert-base")
model = AutoModelForSequenceClassification.from_pretrained("ai-forever/ruBert-base")

# Copy the model to the GPU.
model.to(device)

config.json:   0%|          | 0.00/590 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/716M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [13]:
# Считаем максимальое количество токенов в предложении
max_len = 0

# For every sentence...
for sent in train_X:
    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  751


In [14]:
# больше максимальной длины для ruBert, выставляем максимальную
max_len = 512

In [15]:
batch_size = 8

train_ds = TrainDataset(pd.Series(train_X), pd.Series(train_y), max_len)
train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

eval_ds = TrainDataset(pd.Series(val_X), pd.Series(val_y), max_len)
eval_dataloader = DataLoader(eval_ds, batch_size=batch_size, shuffle=True)

test_ds = TestDataset(pd.Series(test_X), max_len)
test_dataloader = DataLoader(test_ds, batch_size=batch_size)

In [16]:
# Задаем optimizer и sheduler, которые помогут нам с файнтьюном.

optimizer = Adam(model.parameters(), lr=5e-6)

num_epochs = 7
total_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,  # Default value in run_glue.py
    num_training_steps=total_steps
)

In [17]:
train_model(train_dataloader, num_epochs)

Epoch 1 / 7 
 -------------------
Loss train: 0.595087  [    0/ 1749]
Evaluating...
F1-score = 0.422936

Loss train: 0.579012  [  400/ 1749]
Evaluating...
F1-score = 0.334144

Loss train: 0.676667  [  800/ 1749]
Evaluating...
F1-score = 0.336308

Loss train: 0.520890  [ 1200/ 1749]
Evaluating...
F1-score = 0.382348

Loss train: 0.639032  [ 1600/ 1749]
Evaluating...
F1-score = 0.454036

Epoch 2 / 7 
 -------------------
Loss train: 0.716132  [    0/ 1749]
Evaluating...
F1-score = 0.416929

Loss train: 0.702682  [  400/ 1749]
Evaluating...
F1-score = 0.499425

Loss train: 0.667957  [  800/ 1749]
Evaluating...
F1-score = 0.496205

Loss train: 0.479115  [ 1200/ 1749]
Evaluating...
F1-score = 0.514207

Loss train: 0.335801  [ 1600/ 1749]
Evaluating...
F1-score = 0.523709

Epoch 3 / 7 
 -------------------
Loss train: 0.517238  [    0/ 1749]
Evaluating...
F1-score = 0.497302

Loss train: 0.287193  [  400/ 1749]
Evaluating...
F1-score = 0.630362

Loss train: 0.399263  [  800/ 1749]
Evaluating

In [18]:
# Сохраняем модель
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './drive/MyDrive/HW5/ruBert-DaNetQA/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model to ./drive/MyDrive/HW5/ruBert-DaNetQA/


('./drive/MyDrive/HW5/ruBert-DaNetQA/tokenizer_config.json',
 './drive/MyDrive/HW5/ruBert-DaNetQA/special_tokens_map.json',
 './drive/MyDrive/HW5/ruBert-DaNetQA/vocab.txt',
 './drive/MyDrive/HW5/ruBert-DaNetQA/added_tokens.json',
 './drive/MyDrive/HW5/ruBert-DaNetQA/tokenizer.json')

In [19]:
y_pred, _ = test_model(test_dataloader, eval=False)

In [20]:
output = ["true" if i == 1 else "false"  for i in y_pred]
output = [f'{{"idx": {n}, "label": "{i}"}}' for n, i in enumerate(output)]

In [ ]:
#output

In [22]:
with open('./drive/MyDrive/HW5/DaNetQA.jsonl', 'w') as f:
    f.writelines('\n'.join(output))

In [ ]:
# Для загрузки модели
#tokenizer = AutoTokenizer.from_pretrained("./drive/MyDrive/HW5/ruBert-DaNetQA")
#model = AutoModelForSequenceClassification.from_pretrained("./drive/MyDrive/HW5/ruBert-DaNetQA")

# Copy the model to the GPU.
#model.to(device)

In [20]:
max_len = 512

In [23]:
get_reply("Осталась ли библия неизменной с момента создания?",
'''10 сентября 1750 года Синод доложил императрице, что перевод готов для печати.
18 декабря 1751 года Елизаветинская Библия вышла из печати. Все изменения, внесённые при исправлении перевода,
были оговорены, примечания к тексту составили отдельный том, практически равный по объёму тексту самой Библии.
Первый тираж быстро разошёлся, и в 1756 году вышло его второе издание с дополнительными примечаниями на полях и гравюрами,
в котором иеромонах Гедеон  исправил ошибки и опечатки первого издания. В дальнейшем Русская церковь продолжила использовать
в богослужебной практике Елизаветинскую Библию, внеся в неё лишь некоторые несущественные изменения.''')


Осталась ли библия неизменной с момента создания? [0] 0.836887


In [27]:
del model
torch.cuda.empty_cache()

## ruBert(tiny) - 2 версия

In [14]:
## https://huggingface.co/cointegrated/rubert-tiny2
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-tiny2")

# Copy the model to the GPU.
model.to(device)

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, 

In [17]:
# Считаем максимальое количество токенов в предложении
max_len = 0

# For every sentence...
for sent in train_X:
    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  760


In [17]:
# У ruBert-tiny2 - максимальная длина 2048!
# больше максимальной длины для ruBert, выставляем максимальную
# max_len = 512

In [18]:
batch_size = 16

train_ds = TrainDataset(pd.Series(train_X), pd.Series(train_y), max_len)
train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

eval_ds = TrainDataset(pd.Series(val_X), pd.Series(val_y), max_len)
eval_dataloader = DataLoader(eval_ds, batch_size=batch_size, shuffle=True)

test_ds = TestDataset(pd.Series(test_X), max_len)
test_dataloader = DataLoader(test_ds, batch_size=batch_size)

In [19]:
# Задаем optimizer и sheduler, которые помогут нам с файнтьюном.

optimizer = Adam(model.parameters(), lr=5e-6)  # lr - больше чем в обычной модели, иначе очень долго сходиться

num_epochs = 10
total_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,  # Default value in run_glue.py
    num_training_steps=total_steps
)

In [20]:
train_model(train_dataloader, num_epochs)

Epoch 1 / 10 
 -------------------
Loss train: 0.693747  [    0/ 1749]
Evaluating...
F1-score = 0.554907

Loss train: 0.646559  [  800/ 1749]
Evaluating...
F1-score = 0.334144

Loss train: 0.598612  [ 1600/ 1749]
Evaluating...
F1-score = 0.334144

Epoch 2 / 10 
 -------------------
Loss train: 0.617287  [    0/ 1749]
Evaluating...
F1-score = 0.334144

Loss train: 0.622850  [  800/ 1749]
Evaluating...
F1-score = 0.344917

Loss train: 0.620106  [ 1600/ 1749]
Evaluating...
F1-score = 0.506493

Epoch 3 / 10 
 -------------------
Loss train: 0.503280  [    0/ 1749]
Evaluating...
F1-score = 0.539397

Loss train: 0.579075  [  800/ 1749]
Evaluating...
F1-score = 0.643726

Loss train: 0.634786  [ 1600/ 1749]
Evaluating...
F1-score = 0.640216

Epoch 4 / 10 
 -------------------
Loss train: 0.478972  [    0/ 1749]
Evaluating...
F1-score = 0.645940

Loss train: 0.424503  [  800/ 1749]
Evaluating...
F1-score = 0.653694

Loss train: 0.399294  [ 1600/ 1749]
Evaluating...
F1-score = 0.666782

Epoch 5 

In [21]:
# Сохраняем модель
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './drive/MyDrive/HW5/ruBert-tiny2-DaNetQA/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model to ./drive/MyDrive/HW5/ruBert-tiny2-DaNetQA/


('./drive/MyDrive/HW5/ruBert-tiny2-DaNetQA/tokenizer_config.json',
 './drive/MyDrive/HW5/ruBert-tiny2-DaNetQA/special_tokens_map.json',
 './drive/MyDrive/HW5/ruBert-tiny2-DaNetQA/vocab.txt',
 './drive/MyDrive/HW5/ruBert-tiny2-DaNetQA/added_tokens.json',
 './drive/MyDrive/HW5/ruBert-tiny2-DaNetQA/tokenizer.json')

In [22]:
y_pred, _ = test_model(test_dataloader, eval=False)

In [23]:
output = ["true" if i == 1 else "false"  for i in y_pred]
output = [f'{{"idx": {n}, "label": "{i}"}}' for n, i in enumerate(output)]

In [24]:
with open('./drive/MyDrive/HW5/DaNetQA-tiny2.jsonl', 'w') as f:
    f.writelines('\n'.join(output))

In [25]:
get_reply("Осталась ли библия неизменной с момента создания?",
'''10 сентября 1750 года Синод доложил императрице, что перевод готов для печати.
18 декабря 1751 года Елизаветинская Библия вышла из печати. Все изменения, внесённые при исправлении перевода,
были оговорены, примечания к тексту составили отдельный том, практически равный по объёму тексту самой Библии.
Первый тираж быстро разошёлся, и в 1756 году вышло его второе издание с дополнительными примечаниями на полях и гравюрами,
в котором иеромонах Гедеон  исправил ошибки и опечатки первого издания. В дальнейшем Русская церковь продолжила использовать
в богослужебной практике Елизаветинскую Библию, внеся в неё лишь некоторые несущественные изменения.''')


Осталась ли библия неизменной с момента создания? [0] 0.5698089


In [21]:
del model
torch.cuda.empty_cache()